In [1]:
import numpy as np
import pandas as pd
import sys,os
import random
import copy
from time import time

import matplotlib.pyplot as plt
import seaborn as sns


sys.path.append('/home/olya/TUM/DESMOND/DESMOND2/')

from run_unpast import run

from utils.method import read_bic_table
from utils.eval import calculate_perfromance

### Reading annotation and expression,
prepareing inputs for calculate_perfromance()

In [2]:
exprs_file = "exprs/asthma_GSE4302.RMA.maxRowVariance.log2exprs.tsv"
exprs = pd.read_csv(exprs_file, sep="\t",index_col=0)

anno  = pd.read_csv("exprs/asthma_GSE4302.annot.tsv",sep="\t",index_col=0)
anno.head(3)

,asthma,individual,is_smoker,Flovent,sample type:ch1,Th2 asthma
GSM98141,1,01SAGEA,0,0,Asthmatic at baseline,0
GSM98142,1,01SAGEA,0,1,Asthmatic after Flovent,0
GSM98143,1,02SAGEA,0,0,Asthmatic after Placebo,0


In [3]:
known_groups = {"Th2 asthma":{"Th2-high": set(anno.loc[anno["Th2 asthma"]==1,:].index.values)}}
#known_groups["Th2 asthma"]["Th2-low"] = set(anno.index.values).difference(known_groups["Th2 asthma"]["Th2-high"])
known_groups["Th2 asthma"].keys()

dict_keys(['Th2-high'])

In [4]:
anno.shape[0]*0.4, anno["Th2 asthma"].sum()

(47.2, 37)

In [5]:
len(known_groups["Th2 asthma"]["Th2-high"] )

37

### UnPaSt example

In [6]:
unpast_biclusters = read_bic_table("../../bicluster_examples/asthma_GSE4302.seed=42.consensus.biclusters.tsv")
unpast_biclusters.loc[[0,1,59],:]

,SNR,n_genes,n_samples,genes,samples,genes_up,genes_down,gene_indexes,sample_indexes,direction,detected_n_times
id,,,,,,,,,,,
0,7.693274,8,58,"{RPS4Y1, ARSL, TTTY14, KDM5D, DDX3Y, ZFY, UTY,...","{GSM98255, GSM98167, GSM98252, GSM98221, GSM98...","{RPS4Y1, ARSL, TTTY14, KDM5D, DDX3Y, ZFY, UTY,...",{},"{7521, 3781, 17737, 16813, 1006, 13549, 4440, ...","{6, 7, 16, 17, 22, 23, 26, 27, 28, 29, 30, 31,...",UP,5
1,3.690024,17,11,"{CCNC, SLC26A2, SMARCC2, PKN2, TNKS2, ZBTB43, ...","{GSM98224, GSM98189, GSM98200, GSM98204, GSM98...","{CCNC, SLC26A2, SMARCC2, PKN2, TNKS2, ZBTB43, ...",{},"{7232, 17505, 7490, 16799, 14409, 10505, 17611...","{64, 48, 83, 116, 117, 115, 58, 59, 61, 62, 63}",UP,6
59,1.883604,3,37,"{CDH26, POSTN, SERPINB2}","{GSM98149, GSM98215, GSM98167, GSM98193, GSM98...","{CDH26, POSTN, SERPINB2}",{},"{2481, 12148, 13998}","{4, 6, 7, 8, 9, 11, 12, 13, 16, 17, 22, 26, 30...",UP,4


In [7]:
performance, best_matches = calculate_perfromance(unpast_biclusters, known_groups, set(anno.index.values))
performance

Th2 asthma    0.897436
dtype: float64

In [8]:
best_matches

,J,adj_pval,bm_id,is_enriched,weight,classification
Th2-high,0.897436,0.0,59,True,1.0,Th2 asthma


In [9]:
unpast_biclusters.loc[59,:]

SNR                                                          1.883604
n_genes                                                             3
n_samples                                                          37
genes                                        {CDH26, POSTN, SERPINB2}
samples             {GSM98149, GSM98215, GSM98167, GSM98193, GSM98...
genes_up                                     {CDH26, POSTN, SERPINB2}
genes_down                                                         {}
gene_indexes                                     {2481, 12148, 13998}
sample_indexes      {4, 6, 7, 8, 9, 11, 12, 13, 16, 17, 22, 26, 30...
direction                                                          UP
detected_n_times                                                    4
Name: 59, dtype: object

### QUBIC exmaple

In [10]:
qubic_biclusters = pd.read_csv("../../bicluster_examples/GSE4302.qubic.tsv",sep = "\t",index_col=0)
qubic_biclusters['genes'] = qubic_biclusters['genes'].apply(lambda row: eval(row))
qubic_biclusters['samples'] = qubic_biclusters['samples'].apply(lambda row: eval(row))
qubic_biclusters.head(3)

,samples,genes
0,"{GSM98255, GSM98167, GSM98252, GSM98221, GSM98...","{ZNF326, ZIC3, UBIAD1, OR4D2, SNRPD1, KIF3B, S..."
1,"{GSM98255, GSM98167, GSM98252, GSM98221, GSM98...","{ZNF326, ZIC3, UBIAD1, OR4D2, SNRPD1, SLC1A1, ..."
2,"{GSM98255, GSM98167, GSM98252, GSM98221, GSM98...","{ZNF326, ZIC3, UBIAD1, OR4D2, SNRPD1, KIF3B, S..."


In [11]:
performance, best_matches = calculate_perfromance(qubic_biclusters, 
                                                  known_groups, set(anno.index.values),adjust_pvals=True)
performance

Th2 asthma    0.512821
dtype: float64

In [12]:
best_matches

,J,adj_pval,bm_id,is_enriched,weight,classification
Th2-high,0.512821,0.0,934,True,1.0,Th2 asthma


In [13]:
best_matches["adj_pval"]["Th2-high"] 

1.251236007783351e-07

In [14]:
" ".join(qubic_biclusters.loc[934,"samples"])

'GSM98255 GSM98167 GSM98157 GSM98189 GSM98164 GSM98148 GSM98177 GSM98158 GSM98171 GSM98195 GSM98163 GSM98178 GSM98227 GSM98192 GSM98186 GSM98182 GSM98196 GSM98181 GSM98185 GSM98172 GSM98191 GSM98180'

In [15]:
" ".join(sorted(qubic_biclusters.loc[934,"genes"]))

'ADA2 CLCA1 CST1 CST4 DDX3Y DYNC2I1 EIF1AY EOLA1-DT ERVK13-1 FOLR1 IL6R KDM5D LOXL4 MOBP POU2AF1 RAI2 RPS4Y1 SCGB1A1 SLIT2 SNCAIP TTTY14 UTY ZFY'

### k-means example

In [16]:
kmeans_clusters = pd.read_csv("../../bicluster_examples/GSE4302.kmeans_defaults.run_1.tsv.csv",sep ="\t")
kmeans_clusters["samples"] = kmeans_clusters["samples"].apply(lambda row: set(row.split(" ")))
kmeans_clusters 

,samples,n_samples,frac_samples
0,"{GSM98176, GSM98206, GSM98157, GSM98213, GSM98...",46,0.389831
1,"{GSM98239, GSM98179, GSM98234, GSM98196, GSM98...",72,0.610169


In [17]:
performance, best_matches =  calculate_perfromance(kmeans_clusters, known_groups, set(anno.index.values))
performance

Th2 asthma    0.0
dtype: float64

In [18]:
best_matches

,J,adj_pval,bm_id,is_enriched,weight,classification
Th2-high,0.0,NaN,NaN,NaN,1.0,Th2 asthma


In [19]:
#" ".join(kmeans_clusters.loc[0,"samples"])

In [20]:
#" ".join(kmeans_clusters.loc[1,"samples"])